In [7]:
import os
import pandas as pd
import pyprind
import zipfile

In [10]:

# Specifying the path to your zip file and the target directory for extraction
zip_path = 'C:/Users/cantg/Downloads/archive (19).zip'
unzip_dir = 'C:/Users/cantg/Downloads/archive (19)'

# Extracting the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Setting the basepath for the extracted data
basepath = os.path.join(unzip_dir, 'aclImdb')

# Initialize the progress bar and the list for storing data
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
data = []

# Iterating over the dataset to populate the data list
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
                data.append([txt, labels[l]])
                pbar.update()

# Creating a DataFrame from the collected data
df = pd.DataFrame(data, columns=['review', 'sentiment'])


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:07:24


In [11]:
df.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import pickle




In [ ]:
# Load vectorizer and stopwords
with open('vectorizer.py', 'rb') as file:
    vectorizer = pickle.load(file)
with open('stopwords.pkl', 'rb') as file:
    stop_words = pickle.load(file)

# Set up the pipeline with TF-IDF and logistic regression
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, stop_words=stop_words)
param_grid = [{'vect__ngram_range': [(1,1)], 'vect__tokenizer': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(random_state=0, solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)

# Splitting data into training and test sets
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# Fit model
gs_lr_tfidf.fit(X_train, y_train)

print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))